In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_jurors = pd.read_csv("jurors.csv")
data_jurors.head(2)

In [ ]:
data_jurors.shape

In [ ]:
#cant de jurados
print(f'Cantidad de jurados: {len(data_jurors.id.unique())}')

In [ ]:
#cant de juicios
print(f'Cantidad de juicios: {len(data_jurors.trial__id.unique())}')

In [ ]:
# cantidad de jurados por juicio

juror_x_trial = data_jurors.groupby('trial__id').agg(
    count_juror=pd.NamedAgg(column="id", aggfunc="count")
)

# juror_x_trial['count_juror'].hist(bins=30) #--> graficar serie directamente
sns.histplot(data=juror_x_trial, x='count_juror', bins=30)

plt.xlabel('Cantidad de jurados por juicio')
plt.ylabel('Frecuencia')
plt.title('Histograma de la cantidad de jurados por juicio')
plt.show()

In [ ]:
juror_x_trial.sort_values(by='count_juror').describe()

-Promedio de jurados por juicio: 48

-Mínimo de jurados en un juicios : 19

-Máximo de jurados en un juicios : 130


In [ ]:
#descartar valores fuera del rango intercuartílico para observar posible variación en promedio
# q = np.percentile(juror_x_trial, [25, 50, 75])
# mask_interq = (juror_x_trial > q[0.25]) & (juror_x_trial < q[0.75])
# juror_x_trial_iqr = juror_x_trial[mask_interq]
# juror_x_trial_iqr.describe()

In [ ]:
# fig, ax = plt.subplots(nrows=1 , ncols=2, figsize=(10, 5))
# sns.boxplot(data = juror_x_trial, showmeans=True, meanprops={"marker":"o", "markerfacecolor":"cyan", "markersize":"5"}, ax =ax[0])
# ax[0].set_title('Jurados por Juicio')
# sns.boxplot(data = juror_x_trial_iqr, showmeans=True, meanprops={"marker":"o", "markerfacecolor":"cyan", "markersize":"5"}, ax =ax[1] )
# ax[1].set_title('Jurados por Juicio (IQR)')

# ymax = max(juror_x_trial['count_juror'].max(), juror_x_trial_iqr['count_juror'].max())
# ax[0].set_ylim(0, ymax)
# ax[1].set_ylim(0, ymax)

In [ ]:
list(data_jurors.strike_eligibility.unique())

In [ ]:
print(data_jurors.strike_eligibility.value_counts())
print(f'Valores nulos ' , data_jurors.strike_eligibility.isna().sum())
print(f'Total de datos ', data_jurors.strike_eligibility.shape[0])

sns.countplot(x='strike_eligibility', data=data_jurors)

In [ ]:
print(f'Valores nulos ' , data_jurors.struck_by.isna().sum())
print(f'Total de datos ', data_jurors.struck_by.shape[0])

sns.countplot(y='struck_by', data=data_jurors)

print(data_jurors.struck_by.value_counts())

In [ ]:
data_jurors.info()

In [ ]:
data_trials = pd.read_csv("trials.csv")
data_trials.head(5)

In [ ]:
import plotly.express as px

fig = px.histogram(data_trials, x="defendant_race", title="Distribución de la raza de los acusados", color_discrete_sequence=['#1f77b4'])
fig.show()

In [ ]:
data_trials.info()

In [ ]:
data_voir_dire_answers = pd.read_csv("voir_dire_answers.csv")
data_voir_dire_answers.head(1)

In [ ]:
data_voir_dire_answers.info()

In [ ]:
#unión jueces con juicios

merged_jurors_trials = pd.merge(data_jurors, data_trials, left_on='trial__id', right_on='id')
print(merged_jurors_trials.shape)

In [ ]:
merged_jurors_trials.columns

In [ ]:
print(f'Jurados:',data_jurors.columns)
print(f'Juicios:',data_trials.columns)
print(f'Encuesta:',data_voir_dire_answers.columns)

In [ ]:
# busqueda de columnas en comun
common_cols = set(data_jurors.columns).intersection(set(data_trials.columns)).intersection(set(data_voir_dire_answers.columns))
common_cols

In [ ]:
import networkx as nx

# Crear el grafo
G = nx.Graph()

# Agregar nodos (tablas)
G.add_node('Jurors')
G.add_node('Trials')
G.add_node('Survey')

# Agregar conexiones (columnas)
G.add_edge('Jurors', 'Trials', column='trial__id')
G.add_edge('Survey', 'Jurors', column='juror_id')

# Dibujar el grafo
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)

# Agregar etiquetas a las conexiones (columnas)
labels = {('Jurors', 'Trials'): 'trial__id', ('Survey', 'Jurors'): 'juror_id'}
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

plt.show()


-id_x ---> id jurado

-id_y ---> id juicio

-id ---> id cuestionario

In [ ]:
#unión merge con voir

#para ver todas las columnas
# pd.set_option('display.max_columns', None)

#merge con todos los jurados aunque no conozcamos sus datos de encuesta ---> 14875
# data = pd.merge(merged_data , data_voir_dire_answers, left_on='id_x', right_on='juror_id', how = 'outer')
#merge con todos los jurados con sus datos de encuesta ---> 3545
data = pd.merge(merged_jurors_trials , data_voir_dire_answers, left_on='id_x', right_on='juror_id')
print(data.shape)

data.head(5)

In [ ]:
# Juror chosen to serve on jury    3640
# Juror not struck                 3436
# Struck by the defense            2301
# Struck for cause                 2042
# Struck by the state              1852
# Struck without notation           571
# Juror excused/absent              371
# Juror chosen as alternate         370
# Unknown                           291

etiquetas = { 
             'Juror chosen to serve on jury': 1,
             'Juror not struck':  1,
             'Struck by the defense':  0,
             'Struck for cause':  0,
             'Struck by the state':  0,
             'Struck without notation': 0,
             'Juror excused/absent': 0,
             'Juror chosen as alternate': 1,
             'Unknown': 0
             }


#EMPROLIJAr!!

data['struck_by_1'] = data['struck_by'].map(etiquetas)
data['struck_by'] = data['struck_by_1']
data.drop(['struck_by_1'], axis=1, inplace=True)
data

In [ ]:
# pd.set_option('display.max_rows', None)
data.isnull().sum().sort_values(ascending=False)

In [ ]:
data_1 = data.dropna(thresh=.15, axis= 1)
data_1.shape

In [ ]:
data_voir_dire_answers.head()

In [ ]:
data_1 = pd.get_dummies(data_voir_dire_answers, drop_first=True)
data_1.head()

In [ ]:
print(data_voir_dire_answers.shape)
print(round(data_voir_dire_answers.married.value_counts()/data.shape[0],4))
print()
print(round(data_voir_dire_answers.children.value_counts()/data.shape[0],4))
print()
print(round(data_voir_dire_answers.religious.value_counts()/data.shape[0],4))
print()
print(round(data_voir_dire_answers.education.value_counts()/data.shape[0],4))